In [1]:
####################### Create a Customer Travel Destinations Map #######################

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hilo,US,19.7297,-155.0900,61.00,67,1,6.91,clear sky
1,1,Henties Bay,NaN,-22.1160,14.2845,66.99,83,17,3.00,few clouds
2,2,Bengkulu,ID,-3.8004,102.2655,76.86,85,78,6.55,broken clouds
3,3,Jining,CN,35.4050,116.5814,49.68,44,10,5.99,clear sky
4,4,Albany,US,42.6001,-73.9662,35.01,39,76,4.00,light snow


In [4]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature? "))
max_temp = float(input("What is the maximum temperature? "))

What is the minimum temperature? 75
What is the maximum temperature? 90


In [5]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Bengkulu,ID,-3.8004,102.2655,76.86,85,78,6.55,broken clouds
5,5,Saint-Philippe,RE,-21.3585,55.7679,84.20,70,75,6.91,light rain
7,7,Avarua,CK,-21.2078,-159.7750,77.00,88,20,3.44,light rain
10,10,Atuona,PF,-9.8000,-139.0333,80.29,76,60,22.68,broken clouds
11,11,Rikitea,PF,-23.1203,-134.9692,77.47,74,100,13.20,overcast clouds
17,17,Yulara,AU,-25.2406,130.9889,80.60,29,0,9.22,clear sky
25,25,Am Timan,TD,11.0297,20.2827,89.56,12,41,6.35,scattered clouds
31,31,Rio Grande,BR,-32.0350,-52.0986,77.00,65,40,10.36,scattered clouds
35,35,Bontang,ID,0.1333,117.5000,79.90,82,97,8.39,overcast clouds
40,40,Vaini,TO,-21.2000,-175.2000,77.00,100,75,4.61,broken clouds


In [6]:
# Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                214
City                   214
Country                209
Lat                    214
Lng                    214
Max Temp               214
Humidity               214
Cloudiness             214
Wind Speed             214
Current Description    214
dtype: int64

In [10]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna() 

In [11]:
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [13]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Bengkulu,ID,76.86,broken clouds,-3.8004,102.2655,
5,Saint-Philippe,RE,84.20,light rain,-21.3585,55.7679,
7,Avarua,CK,77.00,light rain,-21.2078,-159.7750,
10,Atuona,PF,80.29,broken clouds,-9.8000,-139.0333,
11,Rikitea,PF,77.47,overcast clouds,-23.1203,-134.9692,
17,Yulara,AU,80.60,clear sky,-25.2406,130.9889,
25,Am Timan,TD,89.56,scattered clouds,11.0297,20.2827,
31,Rio Grande,BR,77.00,scattered clouds,-32.0350,-52.0986,
35,Bontang,ID,79.90,overcast clouds,0.1333,117.5000,
40,Vaini,TO,77.00,broken clouds,-21.2000,-175.2000,


In [14]:
# Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    #Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
 
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
# Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Bengkulu,ID,76.86,broken clouds,-3.8004,102.2655,Grage Hotel Bengkulu
5,Saint-Philippe,RE,84.20,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
7,Avarua,CK,77.00,light rain,-21.2078,-159.7750,Paradise Inn
10,Atuona,PF,80.29,broken clouds,-9.8000,-139.0333,Villa Enata
11,Rikitea,PF,77.47,overcast clouds,-23.1203,-134.9692,Pension Maro'i


In [16]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!= '']
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Bengkulu,ID,76.86,broken clouds,-3.8004,102.2655,Grage Hotel Bengkulu
5,Saint-Philippe,RE,84.20,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
7,Avarua,CK,77.00,light rain,-21.2078,-159.7750,Paradise Inn
10,Atuona,PF,80.29,broken clouds,-9.8000,-139.0333,Villa Enata
11,Rikitea,PF,77.47,overcast clouds,-23.1203,-134.9692,Pension Maro'i


In [20]:
# Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_Vacation.csv"

# Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [58]:
# Rename the column "Current Description" to "Current Weather".
clean_hotel_df.columns = ["City","Country","Max Temp","Current Weather","Lat","Lng","Hotel Name"]
clean_hotel_df.head()

,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
2,Bengkulu,ID,76.86,broken clouds,-3.8004,102.2655,Grage Hotel Bengkulu
5,Saint-Philippe,RE,84.20,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
7,Avarua,CK,77.00,light rain,-21.2078,-159.7750,Paradise Inn
10,Atuona,PF,80.29,broken clouds,-9.8000,-139.0333,Villa Enata
11,Rikitea,PF,77.47,overcast clouds,-23.1203,-134.9692,Pension Maro'i


In [53]:
# Using the template add hotel name, the city name, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Weather}</dd>  
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Get the data from each row and add it to the formatting template and store the data in a list.
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [54]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Display the figure
fig

Figure(layout=FigureLayout(height='420px'))